<a href="https://colab.research.google.com/github/giovxna/educa.lab/blob/main/educa_lab_Imers%C3%A3o_IA_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai google-adk

In [ ]:
import os
import textwrap
import warnings
from datetime import date
from google.colab import userdata
from IPython.display import display, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

In [ ]:
# Configuração da API
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
client = genai.Client()

In [ ]:
warnings.filterwarnings("ignore")
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Função auxiliar para executar agente

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

In [ ]:
def criar_agente_explicador(modo_usuario):
    instrucoes = {
        "infantil": "Explique como se o usuário tivesse 5 anos. Use analogias com brinquedos, animais e situações do cotidiano de uma criança.",
        "ensino_medio": "Explique como para um estudante do ensino médio, com exemplos aplicados, mas sem termos muito técnicos.",
        "adulto": "Explique de forma clara, objetiva e técnica, como para um adulto curioso.",
        "fofoca": "Explique como se fosse uma fofoca entre amigos. Use humor, gírias leves e analogias engraçadas como se estivesse contando um babado.",
        "memes": "Explique com tom de meme e linguagem da internet. Use emojis, hashtags e expressões engraçadas."
    }

    return Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction=instrucoes.get(modo_usuario, "Explique de forma clara e divertida."),
        description="Agente que explica conceitos com o estilo desejado."
    )

def agente_quiz(assunto):
    quiz = Agent(
        name="agente_quiz",
        model=MODEL_ID,
        instruction="Crie uma pergunta ou desafio simples para testar se o usuário aprendeu o conteúdo.",
        description="Agente que cria um quiz ou desafio."
    )
    return call_agent(quiz, f"Crie uma pergunta para testar o conhecimento sobre: {assunto}")

def agente_link_alura(assunto):
    link = Agent(
        name="agente_alura",
        model=MODEL_ID,
        instruction="Sugira um curso da Alura que aprofunde esse tema.",
        description="Agente que sugere cursos da Alura."
    )
    return call_agent(link, f"Sugira um curso da Alura para aprender mais sobre: {assunto}")

In [ ]:
def educa_lab(assunto, tempo, modo):
    explicador = criar_agente_explicador(modo)
    prompt = f"Explique o conceito de '{assunto}' em no máximo {tempo} minutos."
    explicacao = call_agent(explicador, prompt)
    quiz = agente_quiz(assunto)
    link = agente_link_alura(assunto)

    print("\n📚 Explicação personalizada:\n")
    display(to_markdown(explicacao))

    print("\n❓ Pergunta rápida para testar aprendizado:\n")
    display(to_markdown(quiz))

    print("\n📎 Para saber mais:\n")
    display(to_markdown(link))

# ---------- Execução principal ---------- #

def iniciar_educa_lab():
    while True:
        assunto = input("\nQuero aprender sobre: ")
        tempo = input("Tempo que tenho (2, 5, 10, 15 minutos): ")

        print("\n🧠 Como você quer aprender?")
        print("1 - Como se eu tivesse 5 anos 👶")
        print("2 - Como estudante do ensino médio 🧑")
        print("3 - Como adulto curioso 👩‍🎓")
        print("4 - Com analogias e fofoca 🤪")
        print("5 - Em memes 😂")
        print("9 - Surpreenda-me com o estilo! 🎲")

        modo_map = {
            "1": "infantil",
            "2": "ensino_medio",
            "3": "adulto",
            "4": "fofoca",
            "5": "memes"
        }

        modo_escolhido = input("Escolha um número: ")

        if modo_escolhido == "9":
            import random
            modo = random.choice(list(modo_map.values()))
            print(f"✨ Estilo sorteado: {modo}")
        else:
            modo = modo_map.get(modo_escolhido, "adulto")

        educa_lab(assunto, tempo, modo)

        repetir = input("\n🔁 Quer aprender outro assunto? (s/n): ").lower()
        if repetir != 's':
            print("\nAté a próxima! 👋")
            break

In [ ]:
# Executar o app
iniciar_educa_lab()


🧠 Como você quer aprender?
1 - Como se eu tivesse 5 anos 👶
2 - Como estudante do ensino médio 🧑
3 - Como adulto curioso 👩‍🎓
4 - Com analogias e fofoca 🤪
5 - Em memes 😂
9 - Surpreenda-me com o estilo! 🎲
✨ Estilo sorteado: ensino_medio

📚 Explicação personalizada:



> Olá! Vamos desvendar o que é uma "planta" de forma bem simples e direta, como se estivéssemos conversando no recreio.
> 
> **O que é uma Planta?**
> 
> Imagine uma casa antes de ser construída. O arquiteto faz um desenho mostrando como cada cômodo vai ficar, onde vai ter porta, janela, etc. Esse desenho é a "planta" da casa. Ela serve como um guia para quem vai construir, mostrando o que deve ser feito e onde.
> 
> No mundo da engenharia e arquitetura, "planta" é basicamente um desenho técnico que representa um lugar (um terreno, um prédio, uma casa) visto de cima. É como se você estivesse voando de helicóptero e olhando para baixo:
> 
> *   **Mostra as medidas:** A planta indica o tamanho de cada parte do lugar, como o comprimento das paredes, a largura dos cômodos, etc.
> *   **Indica o que tem em cada lugar:** A planta mostra onde ficam as paredes, portas, janelas, escadas, pias, vasos sanitários... tudo!
> *   **É um guia:** A planta ajuda a entender como o lugar está organizado e como as coisas se encaixam.
> 
> **Exemplos Práticos**
> 
> 1.  **Planta da sua sala de aula:** Imagine que você quer desenhar a planta da sua sala de aula. Você olha de cima e desenha as paredes, a porta, as janelas, as mesas, a lousa. Na planta, você coloca as medidas de cada parede e a distância entre as mesas. Assim, quem olhar a planta vai saber exatamente como a sala está organizada.
> 
> 2.  **Planta da sua casa:** A planta da sua casa mostra onde ficam os quartos, a sala, a cozinha, o banheiro, etc. Ela também mostra onde estão as portas e janelas. Se você quisesse reformar a casa, a planta seria muito útil para planejar as mudanças.
> 
> 3.  **Planta de um jardim:** Se você quer criar um jardim, pode fazer uma planta para organizar as flores, árvores, caminhos, etc. A planta mostra onde cada planta deve ser colocada e qual o tamanho do jardim.
> 
> **Por que as Plantas são Importantes?**
> 
> As plantas são importantes porque ajudam a planejar e construir coisas de forma organizada e eficiente. Sem uma planta, seria muito difícil construir uma casa ou organizar um jardim, por exemplo.
> 
> **Em Resumo:**
> 
> *   Planta é um desenho técnico que mostra um lugar visto de cima.
> *   Ela indica as medidas e o que tem em cada lugar.
> *   É um guia para planejar e construir coisas.
> 
> Espero que essa explicação tenha sido útil! Se tiver mais alguma dúvida, é só perguntar!



❓ Pergunta rápida para testar aprendizado:



> Qual das seguintes opções **não** é uma parte essencial da maioria das plantas?
> 
> a) Raiz
> b) Caule
> c) Flor
> d) Antena
> 



📎 Para saber mais:



> Para aprofundar seus conhecimentos sobre plantas, recomendo o curso **"Biologia: botânica"** da Alura.
> 
> Este curso oferece uma visão geral sobre o mundo das plantas, desde a sua estrutura e fisiologia até a sua reprodução e evolução. Você irá explorar os principais grupos de plantas, como briófitas, pteridófitas, gimnospermas e angiospermas, e aprender sobre suas características distintas e adaptações ao ambiente.
> 
> Além disso, o curso aborda temas como fotossíntese, respiração, transporte de água e nutrientes, hormônios vegetais e interações ecológicas das plantas. Ao final do curso, você terá uma base sólida em botânica para entender o papel fundamental das plantas nos ecossistemas e na vida humana.
> 
> Para complementar seus estudos, você também pode conferir outros cursos da Alura relacionados à área de biologia e meio ambiente.
